## Multi Models

In [ ]:
# mount drive first
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

In [ ]:
!pip install timm
!pip install transformers timm ftfy

In [ ]:
# u have to get ride of the empty files or the model wont run
import os

folders = [
    "/content/train/ai",
    "/content/train/real",
    "/content/test/ai",
    "/content/test/real",
    "/content/validation/ai",
    "/content/validation/real"
]

count_removed = 0
for folder in folders:
  for f in os.listdir(folder):
    path = os.path.join(folder, f)
    if os.path.isfile(path) and f.lower().endswith(('.jpg', '.jpeg')):
        if os.path.getsize(path) == 0:
            os.remove(path)
            count_removed += 1
            # print(f"Deleted empty file: {path}")
print('Count of deleted files : ', count_removed)


In [ ]:
# imports
import os, sys,torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
size_check = {}
# verify that the images are the same size or not
# print(os.listdir('1_fake')[0:10])
for img in os.listdir('/content/train/ai'):
    if '.ipynb_checkpoints' == img:
        continue
    images=Image.open('/content/train/ai/'+img)
    # print(images.size)
    if images.size in size_check:
        size_check[images.size] += 1
    else:
        size_check[images.size] = 1
# sizes are not the same
print(list(size_check.items())[0:10])


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_path = "/content/train"
test_path  = "/content/validation"

train_data = datasets.ImageFolder(train_path, transform=transform)
test_data  = datasets.ImageFolder(test_path,  transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader  = DataLoader(test_data, batch_size=32, shuffle=False)

class_names = train_data.classes
num_classes = len(class_names)
num_classes

In [ ]:
def train(model, criterion, optimizer, train_loader, val_loader, epochs=10, model_name="model"):
    metrics = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": []
    }

    model.to(device)
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total

        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total

        metrics["train_loss"].append(epoch_train_loss)
        metrics["train_acc"].append(epoch_train_acc)
        metrics["val_loss"].append(epoch_val_loss)
        metrics["val_acc"].append(epoch_val_acc)

        print(
            f"Epoch {epoch+1}/{epochs} "
            f"Train Loss: {epoch_train_loss:.4f}  Train Acc: {epoch_train_acc:.4f} | "
            f"Val Loss: {epoch_val_loss:.4f}  Val Acc: {epoch_val_acc:.4f}"
        )

        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), f"{model_name}.pth")


    plot(model_name, metrics, epochs)

    return model

In [ ]:
def plot(model_name, metrics, epochs):

    x = range(1, epochs + 1)
    plt.figure(figsize=(8, 5))
    plt.plot(x, metrics["train_loss"], label="Train Loss")
    plt.plot(x, metrics["val_loss"],   label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"{model_name} Loss Curve")
    plt.legend()
    plt.savefig(f"{model_name}_loss.png")
    plt.close()


    plt.figure(figsize=(8, 5))
    plt.plot(x, metrics["train_acc"], label="Train Accuracy")
    plt.plot(x, metrics["val_acc"],   label="Val Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(f"{model_name} Accuracy Curve")
    plt.legend()
    plt.savefig(f"{model_name}_accuracy.png")
    plt.close()

In [ ]:
def evaluate(model, model_name="model"):
    model.eval()
    preds = []
    trues = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            preds.extend(predicted.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    acc = accuracy_score(trues, preds)
    prec = precision_score(trues, preds, average="weighted", zero_division=0)
    rec = recall_score(trues, preds, average="weighted", zero_division=0)
    f1 = f1_score(trues, preds, average="weighted", zero_division=0)


    print("Accuracy :", acc)
    print("Precision:", prec)
    print("Recall   :", rec)
    print("F1 Score :", f1)

    return acc, prec, rec, f1


In [ ]:
model = timm.create_model("convnext_tiny", pretrained=True, num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

convnext_model = train(model, criterion, optimizer, train_loader, val_loader, epochs=10, model_name="convnext_tiny")
evaluate(convnext_model, model_name="ConvNeXt Tiny")

In [ ]:
torch.cuda.empty_cache()
model = timm.create_model("vit_large_patch14_224", pretrained=False, num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

vit_model = train(model, criterion, optimizer, train_loader, val_loader, epochs=10, model_name="vit")
evaluate(vit_model, model_name="Vision Transformer")

In [ ]:
torch.cuda.empty_cache()
model = timm.create_model("resnet50", pretrained=True, num_classes=2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

resnet_model = train(model, criterion, optimizer, train_loader, val_loader, epochs=10,model_name="resnet50")
evaluate(resnet_model, model_name="ResNet50")

In [ ]:
torch.cuda.empty_cache()
model = shufflenet_v2_x1_0(weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(1024, 2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

shuffle_model = train(model, criterion, optimizer, train_loader, val_loader, epochs=10, model_name="shufflenetv2")
evaluate(shuffle_model, model_name="ShuffleNetV2")